## Intel & MobileODT Cervical Cancer Screening

I am working on this project for a Kaggle competition:

https://www.kaggle.com/c/intel-mobileodt-cervical-cancer-screening

My goal is accurately identifie a woman’s cervix type (type 1, type 2 or type 3) based on the image. As a rtaning set I am using about 1500 images (about 500 images for every type of cancer).

In [1]:
import tensorflow as tf
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion

In [2]:
TRAIN_DIR1 = 'train/Type_1'
TRAIN_DIR2 = 'train/Type_2'
TRAIN_DIR3 = 'train/Type_3'
TEST_DIR = 'test'
IMG_SIZE = 50 # The photos have different size. I will resize it 50 by 50 just to start (I think, we need to use a better resolution)

In [3]:
training_data = []

In [32]:
pixel_depth = 255.0 
def create_train_data(TRAIN_DIR, label):
    for img in tqdm(os.listdir(TRAIN_DIR)):
        if not img[0].isdigit():
            continue
        path = os.path.join(TRAIN_DIR,img)
        img = (cv2.imread(path,cv2.IMREAD_GRAYSCALE) - pixel_depth / 2) / pixel_depth
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])

In [33]:
create_train_data(TRAIN_DIR1, [1.0,0.0,0.0])
create_train_data(TRAIN_DIR2, [0.0,1.0,0.0])
create_train_data(TRAIN_DIR3, [0.0,0.0,1.0])

100%|██████████| 441/441 [02:06<00:00,  4.07it/s]


In [34]:
len(training_data)

1451

In [35]:
shuffle(training_data)
np.save('train_data_50_50_gray.npy', training_data)

In [4]:
train_data = np.load('train_data_50_50_gray.npy')

In [5]:
train = train_data[:-500]
test = train_data[-500:-250]
valid = train_data[-250: ]

In [6]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = np.array([i[1] for i in train])

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = np.array([i[1] for i in test])

valid_x = np.array([i[0] for i in valid]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
valid_y = np.array([i[1] for i in valid])

In [7]:
test_x = test_x.reshape((-1, IMG_SIZE * IMG_SIZE)).astype(np.float32)
valid_x = valid_x.reshape((-1, IMG_SIZE * IMG_SIZE)).astype(np.float32)
X = X.reshape((-1, IMG_SIZE * IMG_SIZE)).astype(np.float32)

In [9]:
batch_size = 128
image_size = 50
num_labels = 3

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_x)
    tf_test_dataset = tf.constant(test_x)
    #tf_result_dataset = tf.constant(test_pictures)

    num_hidden_nodes = 1554
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    weights2 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights1)+biases1)
    logits = tf.matmul(layer_1, weights2) + biases2
    
    betta = 0.01
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, 
        logits=logits) + betta *(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2)
                                 + tf.nn.l2_loss(biases2)))

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1)+biases1),weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1)+biases1),weights2) + biases2)
    #result = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_result_dataset, weights1)+biases1),weights2) + biases2)

In [10]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [11]:
print('Training set', X.shape, Y.shape)
print('Validation set', valid_x.shape, valid_y.shape)
print('Test set', test_x.shape, test_y.shape)
#print('Result set', test_pictures.shape, test_names.shape)

Training set (951, 2500) (951, 3)
Validation set (250, 2500) (250, 3)
Test set (250, 2500) (250, 3)


In [13]:
num_steps = 2001

with tf.Session(graph=graph) as session:
	tf.initialize_all_variables().run()
	print("Two layers netowrk, Initialized")
	for step in range(num_steps):
		offset = (step * batch_size) % (Y.shape[0] - batch_size)
		batch_data = X[offset:(offset + batch_size), :]
		batch_labels = Y[offset:(offset + batch_size), :]
		feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
		_, l, predictions = session.run(
			[optimizer, loss, train_prediction], feed_dict=feed_dict)
		if (step % 500 == 0):
			print("Minibatch loss at step %d: %f" % (step, l))
			print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
			print("Validation accuracy: %.1f%%" % accuracy(
				valid_prediction.eval(), valid_y))
	print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_y))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Two layers netowrk, Initialized
Minibatch loss at step 0: 15223.503906
Minibatch accuracy: 46.9%
Validation accuracy: 32.0%
Minibatch loss at step 500: 100.955147
Minibatch accuracy: 95.3%
Validation accuracy: 50.0%
Minibatch loss at step 1000: 1.728694
Minibatch accuracy: 60.9%
Validation accuracy: 51.2%
Minibatch loss at step 1500: 0.781542
Minibatch accuracy: 82.0%
Validation accuracy: 44.8%
Minibatch loss at step 2000: 0.968269
Minibatch accuracy: 75.8%
Validation accuracy: 44.4%
Test accuracy: 45.6%


I see that validation accuracy decreases after step 1000 and that test accuracy and validation accuracy are almost equal. Thus, I am goint to try to do 1001 steps only.

In [14]:
num_steps = 1001

with tf.Session(graph=graph) as session:
	tf.initialize_all_variables().run()
	print("Two layers netowrk, Initialized")
	for step in range(num_steps):
		offset = (step * batch_size) % (Y.shape[0] - batch_size)
		batch_data = X[offset:(offset + batch_size), :]
		batch_labels = Y[offset:(offset + batch_size), :]
		feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
		_, l, predictions = session.run(
			[optimizer, loss, train_prediction], feed_dict=feed_dict)
		if (step % 500 == 0):
			print("Minibatch loss at step %d: %f" % (step, l))
			print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
			print("Validation accuracy: %.1f%%" % accuracy(
				valid_prediction.eval(), valid_y))
	print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_y))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Two layers netowrk, Initialized
Minibatch loss at step 0: 15268.683594
Minibatch accuracy: 34.4%
Validation accuracy: 50.8%
Minibatch loss at step 500: 100.725555
Minibatch accuracy: 96.1%
Validation accuracy: 40.0%
Minibatch loss at step 1000: 1.378992
Minibatch accuracy: 89.8%
Validation accuracy: 50.0%
Test accuracy: 47.6%


47.6 is not super good, but is is better than 33% that we can get if we will predit always the same type.

Now, I am going to try increase the picture size up to 100 by 100.